In [ ]:
import random 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Hypothesis test

(a)

Null hypothesis: The coin is fair.

Alternative: No, it’s biased towards heads.

(b)

Null hypothesis: The coin is fair.

Alternative: No, it's not.

In [ ]:
# Simulate 400 flips of a biased coin (P(H)=0.6, P(T)=0.4) and save to CSV
actual_coin = random.choices(['H', 'T'], 
                             k=400, 
                             weights=[0.6, 0.4])
pd.DataFrame(actual_coin, columns=['coin']).to_csv('coins_400.csv', index=False)

In [ ]:
# Pretend we don't know the actual coin bias, and analyze the data
coins_400 = pd.read_csv('coins_400.csv')
sns.histplot(coins_400, x='coin', stat='density')
coins_400.head()

In [ ]:
# (a) percent of heads
heads = (coins_400['coin']=='H')
t1 = sum(heads) / len(coins_400)
t1

In [ ]:
# (b) | percent of heads - 50% | 
t2 = abs(t1 - 0.5)
t2

In [ ]:
# 1. Make a lot of simulated data under the null hypothesis
# 2. Calculate the statistics t1 and t2 from simulated data

statistic = pd.DataFrame(columns=['t1', 't2'])

for simulation in range(1000):
    simulated_coins = pd.DataFrame(random.choices(['H', 'T'], k=400), columns=['coin'])
    heads = (simulated_coins['coin']=='H') # True, False array
    sim_t1 = sum(heads) / len(simulated_coins) # proportion of heads
    sim_t2 = abs(sim_t1-0.5) # difference from 50%
    statistic.loc[len(statistic)] = [sim_t1, sim_t2]
statistic

In [ ]:
# 3a. Plot empirical distribution of t1 (under the null)

fig, ax = plt.subplots(1)
sns.histplot(data=statistic, x='t1', stat='density', label='Under the null')
ax.axvline(t1, color='k', linestyle='-.', label='In dataset')
ax.legend()

In [ ]:
# p1: how many simulated t1s are larger than data t1?
sum(statistic['t1']>=t1)/len(statistic)

In [ ]:
# 3b. Plot empirical distribution of t2 (under the null)
fig, ax = plt.subplots(1)
sns.histplot(data=statistic, x='t2', stat='density', label='Under the null')
ax.axvline(t2, color='k', linestyle='-.', label='In dataset')
ax.legend()

In [ ]:
# p2: how many simulated t2s are larger than data t2?
sum(statistic['t2']>=t2)/len(statistic)

# Hypothesis test for numerical data
Null: Group A and Group B are each sampled from population distributions that are approximately normal with potentially different standard deviations but the same mean.

Alternative: No, B distribution has a higher mean than A distribution.

Statistic: sample mean of B - sample mean of A (or some variation of it)


In [ ]:
# generate two groups of data from normal distributions and save to CSV
mean_A, std_A = 20, 3
mean_B, std_B = 22, 4
n_A = 30
n_B = 20

actual_A = []
for i in range(n_A):
    actual_A.append(random.gauss(mean_A, std_A))

actual_B = []
for i in range(n_B):
    actual_B.append(random.gauss(mean_B, std_B))

pd.DataFrame(actual_A, columns=['value']).to_csv('A.csv', index=False)
pd.DataFrame(actual_B, columns=['value']).to_csv('B.csv', index=False)

In [ ]:
# Pretend we don't know the actual distributions, and analyze the data
group_A = pd.read_csv('A.csv')
group_B = pd.read_csv('B.csv')
group_A.head()

In [ ]:
# one statistic = mean of B - mean of A
group_B['value'].mean() - group_A['value'].mean()


In [ ]:
# t-statistic = (mB - mA)/sqrt(sA^2/nA + sB^2/nB)
import math
top = group_B['value'].mean() - group_A['value'].mean()
bottom = group_A['value'].std()**2/len(group_A) + group_B['value'].std()**2/len(group_B) 
t = top/math.sqrt(bottom)
t

At this point, we can skip running the simulations, because the empirical distribution of t-statistic is known to come from a probability distribution called t-distribution. People have figured out what that distribution looks like and how to calculate the corresponding p-value in the theoretical curve.

`df` stands for degrees of freedom, which is something that depends on number of samples and standard deviations. No need to memorize the formula.

See parameters in 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
from scipy.stats import ttest_ind

ttest_ind(group_A['value'], group_B['value'], 
          equal_var=False, 
          alternative='less') # mean A < mean B
